##Concatenating original dataset's training and validation splits

In [1]:
%pip install transformers==4.28.0 #Installing latest version is running into dependency problems
%pip install datasets
%pip install sentencepiece
%pip install rouge_score
%pip install nbformat
%pip install plotly
%pip install torch
%pip install evaluate
%pip install sklearn
%pip install numpy
%pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 57.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 85.7 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.6/485.6 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 33.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 17.0 MB/s eta 0:00:0

In [2]:
# Login to weights and biases
import wandb
wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [3]:
# Set WANDB project name env variable
%env WANDB_PROJECT=question2rightAnswer
%env WANDB_LOG_MODEL='end'
%env WANDB_WATCH=all


env: WANDB_PROJECT=question2rightAnswer
env: WANDB_LOG_MODEL='end'
env: WANDB_WATCH=all


In [4]:
from datasets import load_dataset,load_metric
# Load dataset from huggingface
dataset = load_dataset("exams", "crosslingual_with_para_pt")
dataset

Generating train split:   0%|          | 0/740 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/184 [00:00<?, ? examples/s]

Dataset exams downloaded and prepared to /root/.cache/huggingface/datasets/exams/crosslingual_with_para_pt/1.0.0/4330a7899d757352941782397b09a4603093ef04e23873967042f5cee03cadd8. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'question', 'answerKey', 'info'],
        num_rows: 740
    })
    validation: Dataset({
        features: ['id', 'question', 'answerKey', 'info'],
        num_rows: 184
    })
})

# Train model with dataset

### Preprocess data

In [5]:
# Set some variables
prefix = "question: "
max_input_length = 512
max_target_length = 32
batch_size = 8
num_train_epochs = 20
# formatacao do modelo mudar se precisar para um padrao
model_name = "question2answer"

In [6]:
from transformers import T5Tokenizer
# modelo base para o finetunning
checkpoint = 'unicamp-dl/ptt5-base-portuguese-vocab'
# inicializa o tokenizer
tokenizer = T5Tokenizer.from_pretrained(checkpoint,model_max_length=max_input_length)

In [7]:
def preprocess_examples(examples):
    """
    example: batch conjunto de exemplos
    Essa funcão é uma das mais importante já que formata nosso texto de entrada tokenizando o texto
    para esse finetunning usamos o dataset de um jeito diferente onde a entrada para o modelo sera uma string com tópico e questão
    e nossa saida esperada(outputs) serão alternativas.
    """
    questions = [x['stem'] for x in dataset['train']['question']]
    subjects = [x['subject'] for x in dataset['train']['info']]
    choices = [x['choices'] for x in dataset['train']['question']]
    correto = list()
    answerAsIndex = list()
    for answerKey in dataset['train']['answerKey']:
      if answerKey=='A':
        answerAsIndex.append(0)
      elif answerKey=='B':
        answerAsIndex.append(1)
      elif answerKey=='C':
        answerAsIndex.append(2)
      else:
        answerAsIndex.append(3)
      correto.append('Sim' if answerKey=='A' else 'Não')
      correto.append('Sim' if answerKey=='B' else 'Não')
      correto.append('Sim' if answerKey=='C' else 'Não')
      correto.append('Sim' if answerKey=='D' else 'Não')
    subjects = [x['subject'] for x in dataset['train']['info']]
    inputs = ["Tópico: {} Questão: {} Contexto: {}".format(
      subjects[i//4],
      questions[i//4],
      choices[i//4]['para'][i%4]
      ) for i in range(4*len(questions)) if correto[i]=='Sim']
    outputs = list()
    for index, choice in enumerate(choices):
        for i in range(4):
            if correto[4*index + i] == "Sim":
                outputs.append(choice['text'][i])
    # inputs e outputs sao texto agora eles vai ser tokenizados lembrar que model_inputs tem diferentes campos ver tutorial 1
    model_inputs = tokenizer(inputs, max_length=max_input_length, padding="max_length", truncation=True)
    # o label ficara só com os tokens_id
    labels = tokenizer(outputs, max_length=max_target_length, padding="max_length", truncation=True).input_ids
    labels_with_ignore_index = []
    for labels_example in labels:
        # devido ao processo de padding é importante não considerar esses ids
        labels_example = [label if label != 0 else -100 for label in labels_example]
        labels_with_ignore_index.append(labels_example)
    model_inputs["labels"] = labels_with_ignore_index
    return model_inputs

In [8]:
# separa o dataset
train,val = dataset['train'],dataset['validation']
# Preprocessa o dataset
encoded_train_ds = train.map(preprocess_examples, batched=True, remove_columns=train.column_names)
encoded_train_ds.set_format(type="torch")
encoded_val_ds = val.map(preprocess_examples, batched=True, remove_columns=val.column_names)
encoded_val_ds.set_format(type="torch")

Map:   0%|          | 0/740 [00:00<?, ? examples/s]

Map:   0%|          | 0/184 [00:00<?, ? examples/s]

In [9]:
from torch.utils.data import DataLoader
def create_dataloaders(train_batch_size=8, eval_batch_size=32):
    train_dataloader = DataLoader(encoded_train_ds, shuffle=True, batch_size=train_batch_size)
    val_dataloader = DataLoader(encoded_val_ds, shuffle=False, batch_size=eval_batch_size)

    return train_dataloader, val_dataloader

In [10]:
from transformers import EarlyStoppingCallback
from transformers import T5Model, T5ForConditionalGeneration

# Carrega o modelo que será usado para o finetunning
model_pt = T5ForConditionalGeneration.from_pretrained(checkpoint)
# Early stopping e uma tecnica onde o finetuning para quando ele nao melhorar depois de 3 vezes consecutivas
early = EarlyStoppingCallback(early_stopping_patience=3)

In [11]:
from transformers import DataCollatorForSeq2Seq

# Inicializa data_collator
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_pt)

In [12]:
from transformers import Seq2SeqTrainingArguments

# Show the training loss with every epoch
logging_steps = len(encoded_train_ds) // batch_size

# Instancia classe Seq2SeqTrainingArguments
args = Seq2SeqTrainingArguments(
    output_dir=f"{model_name}",
    evaluation_strategy="epoch",
    learning_rate=5.6e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    # armazena só um modelo no colab é imporante não usar muita memoria
    save_total_limit=1,
    num_train_epochs=num_train_epochs,
    predict_with_generate=True,
    logging_steps=logging_steps,
    # garante que o modelo armazene o melhor modelo
    load_best_model_at_end=True,
    save_strategy="epoch",
    fp16=True,
    report_to="wandb",
    run_name='raw-dataset'
)

### Set variables and methods related to metrics computation

In [13]:
import nltk
import evaluate
# essa biblioteca é importante para a funcao compute metrics
nltk.download('punkt')

rouge = evaluate.load('rouge', module_type="metric")
bleu = evaluate.load('bleu', module_type="metric")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [14]:
import nltk
import numpy as np

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    # Replace -100s used for padding as we can't decode them
    preds = np.where(preds != -100, preds, tokenizer.pad_token_id)
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Rouge expects a newline after each sentence
    rouge_decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    rouge_decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

    # Some simple post-processing (for BLEU)
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    # Calculo das métricas
    result = rouge.compute(predictions=rouge_decoded_preds, references=rouge_decoded_labels, use_stemmer=True)
    bleu_result = bleu.compute(predictions=decoded_preds, references=decoded_labels)
    result.update({"bleu": bleu_result["bleu"]})

    result = {k: round(v*100, 2) for k, v in result.items()}
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)

    return result

### Finetune the model

In [15]:
from transformers import Seq2SeqTrainer
# Criamos o Objeto trainer que vai receber os argumentos anterios juntos com as entradas numericas
trainer = Seq2SeqTrainer(
    model_pt,
    args,
    train_dataset=encoded_train_ds,
    eval_dataset=encoded_val_ds,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[early]
)

In [16]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [17]:
trainer.train()
wandb.finish()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


wandb: Currently logged in as: hirokibastos (quizzing). Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Bleu,Gen Len
1,5.316100,3.158170,20.840000,7.340000,19.240000,19.220000,3.550000,9.956757
2,3.267000,2.700729,23.430000,9.650000,21.780000,21.750000,4.980000,11.128378
3,2.905700,2.358870,26.540000,12.300000,24.510000,24.470000,6.570000,10.979730
4,2.617300,2.089830,30.260000,14.660000,27.940000,27.930000,10.300000,10.763514
5,2.410300,1.871308,32.170000,16.960000,29.810000,29.740000,11.900000,10.725676
6,2.213200,1.690531,34.680000,18.970000,32.230000,32.200000,14.490000,10.951351
7,2.046800,1.530791,36.310000,20.810000,34.000000,33.980000,16.430000,11.309459
8,1.953900,1.393056,39.080000,23.690000,36.750000,36.750000,18.860000,11.131081
9,1.811800,1.270855,41.220000,26.070000,38.800000,38.730000,20.490000,10.908108
10,1.700100,1.166937,43.250000,28.270000,41.010000,40.960000,22.580000,11.090541


eval/bleu,▁▁▂▃▃▄▄▅▅▆▆▆▆▇▇▇████
eval/gen_len,▁▇▆▅▅▆█▇▆▇▆▆▅▆█▆▆▆▆▆
eval/loss,█▇▆▅▄▄▃▃▂▂▂▂▂▁▁▁▁▁▁▁
eval/rouge1,▁▂▂▃▄▄▄▅▆▆▆▇▇▇██████
eval/rouge2,▁▂▂▃▃▄▄▅▅▆▆▆▇▇██████
eval/rougeL,▁▂▂▃▃▄▄▅▅▆▆▇▇▇██████
eval/rougeLsum,▁▂▂▃▃▄▄▅▅▆▆▇▇▇██████
eval/runtime,▃▄▂▃▄▂█▃▂▂▁▃▃▁▄▅▂▁▅▃
eval/samples_per_second,▆▅▇▆▅▇▁▆▇▇█▆▆█▄▄▇█▄▅
eval/steps_per_second,▆▅▇▆▅▇▁▆▇▆█▆▆█▄▄▇█▄▅
train/epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇████


# Save finetunned model to gdrive

In [18]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [19]:
import torch
model_save_name = 'right_answers_raw_dataset.pt'
path = F"/content/gdrive/MyDrive/Models/{model_save_name}"
torch.save(model_pt.state_dict(), path)

Referencia:

- Metrics
  https://huggingface.co/spaces/evaluate-metric/bleu

  https://huggingface.co/docs/evaluate/transformers_integrations

  https://github.com/huggingface/transformers/blob/main/examples/pytorch/translation/run_translation.py

  https://huggingface.co/spaces/evaluate-metric/bleu

- Balancing dataset
  https://discuss.huggingface.co/t/k-fold-cross-validation/5765/5

  https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedKFold.html

- Model Registry
  https://docs.wandb.ai/guides/integrations/huggingface